# Feature Engineering

The next step is to create features from the raw text so we can train the machine learning models. The steps followed are:

1. **Text Cleaning and Preparation**: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization. 
2. **Label coding**: creation of a dictionary to map each category to a code.
3. **Train-test split**: to test the models on unseen data.
4. **Text representation**: use of TF-IDF scores to represent text.

In [ ]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

First of all we'll load the dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pathcolab='/content/drive/MyDrive/AI Projects/Feature Engineering/ICE 1/Already_Created_Set/Latest-News-Classifier/0. Latest News Classifier/02. Exploratory Data Analysis/'
path_df = 'News_dataset.pickle'
with open(pathcolab+path_df, 'rb') as data:
    df = pickle.load(data)

In [ ]:
df.head()

,File_Name,Content,Category,Complete_Filename,id,News_length
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business,1,2559
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business,1,2251
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business,1,1551
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business,1,2411
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business,1,1569


And visualize one sample news content:

In [ ]:
df.loc[1]['Content']

'Dollar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\n\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of conditions u

## 1. Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [ ]:
# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

Regarding 3rd and 4th bullet, although it seems there is a special character, it won't affect us since it is not a *real* character:

In [ ]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [ ]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

We'll downcase the texts because we want, for example, `Football` and `football` to be the same word.

In [ ]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Punctuation signs

Punctuation signs won't have any predicting power, so we'll just get rid of them.

In [ ]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

By doing this we are messing up with some numbers, but it's no problem since we aren't expecting any predicting power from them.

### 1.4. Possessive pronouns

We'll also remove possessive pronoun terminations:

In [ ]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [ ]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In order to lemmatize, we have to iterate through every word:

In [ ]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [ ]:
df['Content_Parsed_5'] = lemmatized_text_list

Although lemmatization doesn't work perfectly in all cases (as can be seen in the example below), it can be useful.

### 1.6. Stop words

In [ ]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [ ]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

To remove the stop words, we'll handle a regular expression only detecting whole words, as seen in the following example:

In [ ]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

We can now loop through all the stop words:

In [ ]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

We have some dobule/triple spaces between words because of the replacements. However, it's not a problem because we'll tokenize by the spaces later.

As an example, we'll show an original news article and its modifications throughout the process:

In [ ]:
df.loc[5]['Content']

'An autonomous Uber car killed a woman in the street in Arizona, police said, in what appears to be the first reported fatal crash involving a self-driving vehicle and a pedestrian in the US.\nThe company said it was pausing its self-driving car operations in Phoenix, Pittsburgh, San Francisco and Toronto.\nPhotograph: APThe self-driving technology is supposed to detect pedestrians, cyclists and others and prevent crashes.\nSimpson said he was unaware of any previous fatal crashes involving an autonomous vehicle and a pedestrian.\nEarlier this year, California regulators approved the testing of self-driving cars on public roads without human drivers monitoring inside.'

1. Special character cleaning

In [ ]:
df.loc[5]['Content_Parsed_1']

'An autonomous Uber car killed a woman in the street in Arizona, police said, in what appears to be the first reported fatal crash involving a self-driving vehicle and a pedestrian in the US. The company said it was pausing its self-driving car operations in Phoenix, Pittsburgh, San Francisco and Toronto. Photograph: APThe self-driving technology is supposed to detect pedestrians, cyclists and others and prevent crashes. Simpson said he was unaware of any previous fatal crashes involving an autonomous vehicle and a pedestrian. Earlier this year, California regulators approved the testing of self-driving cars on public roads without human drivers monitoring inside.'

2. Upcase/downcase

In [ ]:
df.loc[5]['Content_Parsed_2']

'an autonomous uber car killed a woman in the street in arizona, police said, in what appears to be the first reported fatal crash involving a self-driving vehicle and a pedestrian in the us. the company said it was pausing its self-driving car operations in phoenix, pittsburgh, san francisco and toronto. photograph: apthe self-driving technology is supposed to detect pedestrians, cyclists and others and prevent crashes. simpson said he was unaware of any previous fatal crashes involving an autonomous vehicle and a pedestrian. earlier this year, california regulators approved the testing of self-driving cars on public roads without human drivers monitoring inside.'

3. Punctuation signs

In [ ]:
df.loc[5]['Content_Parsed_3']

'an autonomous uber car killed a woman in the street in arizona police said in what appears to be the first reported fatal crash involving a self-driving vehicle and a pedestrian in the us the company said it was pausing its self-driving car operations in phoenix pittsburgh san francisco and toronto photograph apthe self-driving technology is supposed to detect pedestrians cyclists and others and prevent crashes simpson said he was unaware of any previous fatal crashes involving an autonomous vehicle and a pedestrian earlier this year california regulators approved the testing of self-driving cars on public roads without human drivers monitoring inside'

4. Possessive pronouns

In [ ]:
df.loc[5]['Content_Parsed_4']

'an autonomous uber car killed a woman in the street in arizona police said in what appears to be the first reported fatal crash involving a self-driving vehicle and a pedestrian in the us the company said it was pausing its self-driving car operations in phoenix pittsburgh san francisco and toronto photograph apthe self-driving technology is supposed to detect pedestrians cyclists and others and prevent crashes simpson said he was unaware of any previous fatal crashes involving an autonomous vehicle and a pedestrian earlier this year california regulators approved the testing of self-driving cars on public roads without human drivers monitoring inside'

5. Stemming and Lemmatization

In [ ]:
df.loc[5]['Content_Parsed_5']

'an autonomous uber car kill a woman in the street in arizona police say in what appear to be the first report fatal crash involve a self-driving vehicle and a pedestrian in the us the company say it be pause its self-driving car operations in phoenix pittsburgh san francisco and toronto photograph apthe self-driving technology be suppose to detect pedestrians cyclists and others and prevent crash simpson say he be unaware of any previous fatal crash involve an autonomous vehicle and a pedestrian earlier this year california regulators approve the test of self-driving cars on public roads without human drivers monitor inside'

6. Stop words

In [ ]:
df.loc[5]['Content_Parsed_6']

' autonomous uber car kill  woman   street  arizona police say   appear    first report fatal crash involve  self-driving vehicle   pedestrian   us  company say   pause  self-driving car operations  phoenix pittsburgh san francisco  toronto photograph apthe self-driving technology  suppose  detect pedestrians cyclists  others  prevent crash simpson say   unaware   previous fatal crash involve  autonomous vehicle   pedestrian earlier  year california regulators approve  test  self-driving cars  public roads without human drivers monitor inside'

Finally, we can delete the intermediate columns:

In [ ]:
df.head(1)

,File_Name,Content,Category,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,The Tempe Police Department said it was invest...,We continue to assist investigators in any way...,autonomous car,1,799.0,We continue to assist investigators in any way...,we continue to assist investigators in any way...,we continue to assist investigators in any way...,we continue to assist investigators in any way...,we continue to assist investigators in any way...,continue assist investigators way ”uber ...


In [ ]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [ ]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,The Tempe Police Department said it was invest...,autonomous car,The Tempe Police Department said it was invest...,We continue to assist investigators in any way...,continue assist investigators way ”uber ...
1,"On Sunday, the inevitable happened: An autonom...",autonomous car,"On Sunday, the inevitable happened: An autonom...",Cars don’t see wellAutonomous cars don’t track...,cars ’ see wellautonomous cars ’ track center...
2,Even before a driverless Uber vehicle struck a...,autonomous car,Even before a driverless Uber vehicle struck a...,"The accident in Tempe, Arizona, was believed t...",accident tempe arizona believe first tim...
3,"On Sunday night, a woman died after she was hi...",autonomous car,"On Sunday night, a woman died after she was hi...","On Sunday night, a woman died after she was hi...",sunday night woman die hit self-driving...
4,A self-driving vehicle made by Uber has struck...,autonomous car,A self-driving vehicle made by Uber has struck...,Something unexpectedly entering the vehicle’s ...,something unexpectedly enter vehicle’ path p...


**IMPORTANT:**

We need to remember that our model will gather the latest news articles from different newspapers every time we want. For that reason, we not only need to take into account the peculiarities of the training set articles, but also possible ones that are present in the gathered news articles.

For this reason, possible peculiarities have been studied in the *05. News Scraping* folder.

## 2. Label coding

We'll create a dictionary with the label codification:

In [ ]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

In [ ]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

## 3. Train - test split

We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

Since we don't have much observations (only 2.225), we'll choose a test set size of 15% of the full dataset.

## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

See `TfidfVectorizer?` for further detail.

It needs to be mentioned that we are implicitly scaling our data when representing it as TF-IDF features with the argument `norm`.

In [ ]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

We have chosen these values as a first approximation. Since the models that we develop later have a very good predictive power, we'll stick to these values. But it has to be mentioned that different combinations could be tried in order to improve even more the accuracy of the models.

In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'autonomous car' category:
  . Most correlated unigrams:
. vehicles
. self
. cars
. driving
. autonomous
  . Most correlated bigrams:
. year old
. self driving

# 'business' category:
  . Most correlated unigrams:
. oil
. market
. bank
. growth
. firm
  . Most correlated bigrams:
. year old
. self driving

# 'entertainment' category:
  . Most correlated unigrams:
. tv
. music
. star
. award
. film
  . Most correlated bigrams:
. mr blair
. self driving

# 'politics' category:
  . Most correlated unigrams:
. tory
. blair
. election
. party
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. champion
. coach
. cup
. match
. game
  . Most correlated bigrams:
. self driving
. year old



As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [ ]:
bigrams

['tell bbc', 'last year', 'prime minister', 'mr blair', 'year old', 'say mr']

We can see there are only six. This means the unigrams have more correlation with the category than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [ ]:
PathColab='/content/drive/MyDrive/AI Projects/Feature Engineering/ICE 1/Assignemnt_data/Pickles/'


# X_train
with open(PathColab+'X_test.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open(PathColab+'X_train.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open(PathColab+'y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open(PathColab+'y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open(PathColab+'df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open(PathColab+'features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open(PathColab+'labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open(PathColab+'features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open(PathColab+'labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open(PathColab+'tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)